In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from bs4 import BeautifulSoup
import unicodedata
import contractions
import google_list
import re
import nltk

In [2]:
input_path = '/home/werewolf97/imtech/pro/data/'

In [3]:
input_path = Path(input_path)

In [4]:
text_data = []

In [5]:
input_path

PosixPath('/home/werewolf97/imtech/pro/data')

In [6]:
converted_path = Path.joinpath(input_path.parent, "converted_files")
if not os.path.exists(converted_path):
    os.mkdir(converted_path)
#print(type(self.input_path.as_posix()))
files = input_path.rglob(f"*.pdf")
#print("Files found: {0}".format(files))
exts = [".pdf", ".txt"]

In [7]:
for file in input_path.rglob('*'):
#     path = dirc+file
    with open(file, "r", encoding="utf-8") as f:
        content = f.read().replace('\n', ' ')
    if content is not None:
        data = [f'{file.parent.name}_{file.name}', content]
        text_data.append(data)
text_corpus = pd.DataFrame(text_data, columns=['resource_id', 'description'])

In [8]:
corpus = text_corpus

In [9]:
corpus['full_text'] = ""
corpus['full_text'] = corpus['description']

In [10]:
len(corpus)

67

In [11]:
def strip_html_tags(text):
#     """Strips the HTML Tags from the text"""
    try:
        soup = BeautifulSoup(text, "html.parser")
        stripped_text = soup.get_text()
    except:
        stripped_text = text
    return stripped_text

In [12]:
def remove_accented_chars(text):
#         """Remove Non UTF-8 Characters from text"""
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [13]:
def expand_contractions(text, contraction_mapping=contractions.CONTRACTION_MAP):
#         """Replace contractions in string of text"""
        contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),
                                          flags=re.IGNORECASE | re.DOTALL)

        def expand_match(contraction):
            match = contraction.group(0)
            first_char = match[0]
            expanded_contraction = contraction_mapping.get(match) \
                if contraction_mapping.get(match) \
                else contraction_mapping.get(match.lower())
            expanded_contraction = first_char + expanded_contraction[1:]
            return expanded_contraction

        try:
            expanded_text = contractions_pattern.sub(expand_match, text)
            expanded_text = re.sub("'", "", expanded_text)
        except:
            expanded_text = text
        return expanded_text

In [14]:
def tokenize(text):
#         """Use the ToktokTokenizer to obtain tokens from the text"""
    tokenizer = nltk.tokenize.toktok.ToktokTokenizer()
    tokens = tokenizer.tokenize(text)
    return tokens

In [15]:
def lemmatize_text(text):
#         """Lemmatize the text using the WordNetLemmatizer"""
    tokenizer = nltk.tokenize.toktok.ToktokTokenizer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    tokens = tokenizer.tokenize(text)
    text = ' '.join([lemmatizer.lemmatize(word) for word in tokens])
    return text

In [16]:
def remove_special_characters(text, remove_digits):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

In [17]:
def remove_stopwords( text):
#         """Remove Stopwords defined in the stopwords corpus"""
    tokenizer = nltk.tokenize.toktok.ToktokTokenizer()
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    stopword_list = nltk.corpus.stopwords.words('english')
    stopword_list.remove('no')
    stopword_list.remove('not')
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    tokens = filtered_tokens
    stopword_list = google_list.stopwords
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [18]:
def normalize_corpus(corpus):
    normalized_corpus = []
    counter = 0
    for i in range(len(corpus)):
        doc = corpus['description'][i]
        counter += 1
        doc = strip_html_tags(doc)
        doc = remove_accented_chars(doc)
        doc = expand_contractions(doc)
        doc = doc.lower()
        doc = re.sub(r'[\r|\n|\r\n]+', ' ', doc)
        doc = lemmatize_text(doc)
        special_char_pattern = re.compile(r'([{.(-)!}])')
        doc = special_char_pattern.sub(" \\1 ", doc)
        doc = remove_special_characters(doc, remove_digits= True)
        doc = re.sub(' +', ' ', doc)
        doc = remove_stopwords(doc)
        normalized_corpus.append(doc)
    return normalized_corpus

In [19]:
corpus['clean_text'] = normalize_corpus(corpus)

In [20]:
def get_tokens(corpus):
#         """Obtain Tokens for each document in the corpus"""
#     tokenizer = nltk.tokenize.toktok.ToktokTokenizer()
    tokenized_doc = []
    for doc in corpus:
        tokens = tokenize(doc)
        filtered_tokens = [w for w in tokens if len(w) > 2]
        tokenized_doc.append(filtered_tokens)
    return tokenized_doc

In [21]:
corpus['tokens'] = get_tokens(corpus['clean_text'])

In [22]:
l = corpus['tokens'][0]

In [23]:
corpus

,resource_id,description,full_text,clean_text,tokens
0,data_LNch7.txt,7 Connectivity Frank Kammer and Hanjo T¨aubig...,7 Connectivity Frank Kammer and Hanjo T¨aubig...,kammer hanjo aubig tices edgedisjoint vertices...,"[kammer, hanjo, aubig, tices, edgedisjoint, ve..."
1,data_Mandate2.txt,Network Science for the Web Mandate - 2: St...,Network Science for the Web Mandate - 2: St...,centrality simultaneous latent semantics found...,"[centrality, simultaneous, latent, semantics, ..."
2,data_LNch3.txt,"3 Centrality Indices Dirk Kosch¨utzki,∗ Kathar...","3 Centrality Indices Dirk Kosch¨utzki,∗ Kathar...",centrality dirk kosch utzki katharina lehmann ...,"[centrality, dirk, kosch, utzki, katharina, le..."
3,data_SocialLearningL2.txt,Social Learning in Networks ...,Social Learning in Networks ...,nonbayesian hoc specie interacting andor obser...,"[nonbayesian, hoc, specie, interacting, andor,..."
4,data_NCMch10.txt,CHAPTER 10 Matching Markets We have now seen...,CHAPTER 10 Matching Markets We have now seen...,brae paradox networkstructured implicit encode...,"[brae, paradox, networkstructured, implicit, e..."
...,...,...,...,...,...
62,data_NCMch14.txt,CHAPTER 14 Link Analysis and Web Search 14.1...,CHAPTER 14 Link Analysis and Web Search 14.1...,edu intrinsic [ ] reponse inexpressive suffers...,"[edu, intrinsic, reponse, inexpressive, suffer..."
63,data_LNch12.txt,12 Network Comparison Michael Baur and Marc B...,12 Network Comparison Michael Baur and Marc B...,baur benkert formalize equivalence undirected ...,"[baur, benkert, formalize, equivalence, undire..."
64,data_LNch1.txt,1 Introduction Ulrik Brandes and Thomas Erleb...,1 Introduction Ulrik Brandes and Thomas Erleb...,ulrik brandes erlebach misleading overloaded d...,"[ulrik, brandes, erlebach, misleading, overloa..."
65,data_NetworkCENTRALITYL2.txt,Shapley Value Recap  Cooperating Agents: A...,Shapley Value Recap  Cooperating Agents: A...,shapley recap cooperating shapley distributes ...,"[shapley, recap, cooperating, shapley, distrib..."


In [24]:
dic = {}
for i in l:
    if i in dic.keys():
        dic[i]+=1
    else:
        dic[i]=1

In [25]:
sort = sorted(dic, key=dic.get)

In [26]:
dic

{'kammer': 18,
 'hanjo': 1,
 'aubig': 18,
 'tices': 4,
 'edgedisjoint': 8,
 'vertices': 11,
 'kvertex': 1,
 'kedge': 1,
 'maximal': 9,
 'kconnected': 3,
 'summarize': 1,
 'denote': 9,
 'vertexconnectivity': 11,
 'cannot': 4,
 'undirected': 26,
 'symmetric': 2,
 'parenthesis': 1,
 'articulation': 2,
 'cutedge': 2,
 'isthmus': 1,
 'ponent': 2,
 'biconnected': 25,
 'nonseparable': 2,
 'compo': 2,
 'nent': 1,
 'cutvertex': 10,
 'ilarly': 1,
 'subgraph': 11,
 'chap': 1,
 'ter': 1,
 'disconnect': 2,
 'brandes': 1,
 'erlebach': 1,
 'lncs': 1,
 'cid': 272,
 'springerverlag': 1,
 'heidelberg': 1,
 'vertexdisjoint': 5,
 'vertexcutset': 1,
 'edgecutset': 1,
 'vertexedgedisjoint': 1,
 'vertexedgecutsets': 1,
 'cutedges': 1,
 'subgraphs': 4,
 'blockgraph': 2,
 'correspond': 1,
 'cutpointgraph': 2,
 'cutvertices': 5,
 'reside': 2,
 'blockcutpointgraph': 1,
 'bipartite': 1,
 'blockvertex': 1,
 'blockcutpoint': 1,
 'kvertexconnected': 1,
 'edgeconnected': 1,
 'kvertexcomponents': 2,
 'kedgecomponents'

In [27]:
sort

['hanjo',
 'kvertex',
 'kedge',
 'summarize',
 'parenthesis',
 'isthmus',
 'nent',
 'ilarly',
 'chap',
 'ter',
 'brandes',
 'erlebach',
 'lncs',
 'springerverlag',
 'heidelberg',
 'vertexcutset',
 'edgecutset',
 'vertexedgedisjoint',
 'vertexedgecutsets',
 'cutedges',
 'correspond',
 'blockcutpointgraph',
 'bipartite',
 'blockvertex',
 'blockcutpoint',
 'kvertexconnected',
 'edgeconnected',
 'kedgecomponent',
 'graphtheoretic',
 'lished',
 'intersecting',
 'nchain',
 'narc',
 'stvertex',
 'edgecuts',
 'stedgeseparator',
 'explicitely',
 'elia',
 'feinstein',
 'hassler',
 'deriving',
 'precondition',
 'sion',
 'kedgeconnected',
 'schrijver',
 'mutually',
 'nonintersecting',
 'chartrand',
 'kvertexedgecomponents',
 'kodama',
 'overlap',
 'overlapping',
 'decom',
 'contradict',
 'minit',
 'prof',
 'mader',
 'interrelation',
 'jecture',
 'pseudosymmetric',
 'endpoint',
 'timofeev',
 'assigning',
 'residual',
 'stflow',
 'saturates',
 'stcuts',
 'iff',
 'allpairs',
 'gomory',
 'pacity',
 'r